## Course: Extreme Gradient Boosting with XGBoost </br>
### Section: Using XGBoost in pipelines </br>

#### Kidney disease case study
##### Link to dataset: https://archive.ics.uci.edu/dataset/336/chronic+kidney+disease

In [1]:
from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb

from ucimlrepo import fetch_ucirepo

In [2]:
# fetch dataset
chronic_kidney_disease = fetch_ucirepo(id=336)

X = chronic_kidney_disease.data.features
y = chronic_kidney_disease.data.targets 

In [3]:
print(chronic_kidney_disease.variables)

     name     role         type demographic              description  \
0     age  Feature      Integer         Age                     None   
1      bp  Feature      Integer        None           blood pressure   
2      sg  Feature  Categorical        None         specific gravity   
3      al  Feature  Categorical        None                  albumin   
4      su  Feature  Categorical        None                    sugar   
5     rbc  Feature       Binary        None          red blood cells   
6      pc  Feature       Binary        None                 pus cell   
7     pcc  Feature       Binary        None          pus cell clumps   
8      ba  Feature       Binary        None                 bacteria   
9     bgr  Feature      Integer        None     blood glucose random   
10     bu  Feature      Integer        None               blood urea   
11     sc  Feature   Continuous        None         serum creatinine   
12    sod  Feature      Integer        None                   so

In [4]:
# Create a boolean mask for categorical columns
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64


In [5]:
# Get list of categorical column names
categorical_feature_mask = X.dtypes == object
categorical_columns = X.columns[categorical_feature_mask].tolist()

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

In [6]:
# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
                                            [([numeric_feature], SimpleImputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True,
                                            df_out=True
                                           )

In [7]:
numeric_imputation_mapper

DataFrameMapper(df_out=True, drop_cols=[],
                features=[(['age'], SimpleImputer(strategy='median')),
                          (['bp'], SimpleImputer(strategy='median')),
                          (['sg'], SimpleImputer(strategy='median')),
                          (['al'], SimpleImputer(strategy='median')),
                          (['su'], SimpleImputer(strategy='median')),
                          (['bgr'], SimpleImputer(strategy='median')),
                          (['bu'], SimpleImputer(strategy='median')),
                          (['sc'], SimpleImputer(strategy='median')),
                          (['sod'], SimpleImputer(strategy='median')),
                          (['pot'], SimpleImputer(strategy='median')),
                          (['hemo'], SimpleImputer(strategy='median')),
                          (['pcv'], SimpleImputer(strategy='median')),
                          (['wbcc'], SimpleImputer(strategy='median')),
                          (['rbcc'], SimpleImputer(strategy='median'))],
                input_df=True)

In [8]:
categorical_imputation_mapper = DataFrameMapper(
                                                [(category_feature, SimpleImputer()) for category_feature in categorical_columns],
                                                input_df=True,
                                                df_out=True
                                               )

In [9]:
numeric_categorical_union = FeatureUnion([
                                          ("num_mapper", numeric_imputation_mapper),
                                          ("cat_mapper", categorical_imputation_mapper)
                                         ])

In [10]:
from sklearn.base import TransformerMixin

class Dictifier(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [dict(row) for row in X]

In [11]:
pipeline = Pipeline([
                     ("featureunion", numeric_categorical_union),
                     ("dictifier", Dictifier()),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3))
                    ])

In [12]:
pipeline

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('num_mapper',
                                                 DataFrameMapper(df_out=True,
                                                                 drop_cols=[],
                                                                 features=[(['age'],
                                                                            SimpleImputer(strategy='median')),
                                                                           (['bp'],
                                                                            SimpleImputer(strategy='median')),
                                                                           (['sg'],
                                                                            SimpleImputer(strategy='median')),
                                                                           (['al'],
                                                                            SimpleImputer(strategy='median')),
                                                                           (['su'],
                                                                            SimpleImputer(strategy='median')),
                                                                           (['bgr'],
                                                                            Si...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])